<a href="https://colab.research.google.com/github/Arsalan9002/Information-Processing-Techniques/blob/master/ApacheBeam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!{'pip install apache-beam[gcp]'}

In [17]:
!{'mkdir -p data'}

In [17]:
!ls

data  demo_code.txt  dept_data.txt  output  sample_data


In [18]:
#to import local files into google colab
from google.colab import files
uploaded = files.upload()

Saving data.txt to data.txt


In [ ]:
!cat dept_data.txt

In [ ]:
import apache_beam as beam
p2 = beam.Pipeline()

lines = (
      p2
      | beam.Create(['Arsalan','Mehmood','Systems Limited','Karachi'])
      | beam.io.WriteToText('data/outCreate1')
)

p2.run()

!{('head -n 20 data/outCreate1-00000-of-00001')}

In [ ]:
import apache_beam as beam
p2 = beam.Pipeline()

lines = (
      p2
      | beam.Create({'row1':[1,2,3,4,5],
                     'row2':[6,7,8,9]
                     })
      | beam.io.WriteToText('data/outCreate1')
)

p2.run()

!{('head -n 20 data/outCreate1-00000-of-00001')}

In [ ]:
#create a pipeline to show the attendance count of each employee of finance department
import apache_beam as beam

p1 = beam.Pipeline()

attendance_count = (
    p1
    | 'Reading dept data' >> beam.io.ReadFromText('dept_data.txt')
    | 'Splitting records' >> beam.Map(lambda x: x.split(','))
    | 'Filtering Accounts dept' >> beam.Filter(lambda x: x[3]=='Accounts')
    | 'Creating Key value pairs' >> beam.Map(lambda x: (x[1],1))
    | 'Grouping & counting Keys' >> beam.CombinePerKey(sum)
    | 'Writing to file' >> beam.io.WriteToText('data/output_new_final')
)

p1.run()

!{('head -n 20 data/output_new_final-00000-of-00001')}

In [ ]:
#to view the output file
!cat data/output_new_final-00000-of-00001

In [15]:
#branching pipelines
import apache_beam as beam

with beam.Pipeline() as p:
  input_collection = (
      p
      | 'reading department data' >> beam.io.ReadFromText('dept_data.txt')
      | 'split rows' >> beam.Map(lambda x: x.split(','))
  )

  accounts_attendance = (
      input_collection
      | 'filter for accounts' >> beam.Filter(lambda x: x[3]=='Accounts')
      | 'pair each accounts employee with 1' >> beam.Map(lambda x: ("Accounts, " + x[1],1))
      | 'Grouping and Sum1' >> beam.CombinePerKey(sum)
  )

  hr_attendance = (
      input_collection
      | 'filter for HR' >> beam.Filter(lambda x: x[3]=='HR')
      | 'pair each HR employee with 1' >> beam.Map(lambda x: ("HR, "+x[1],1))
      | 'Grouping and Sum' >> beam.CombinePerKey(sum)
  )

  output = (
      (accounts_attendance, hr_attendance)
      | 'Merging two PCollections' >> beam.Flatten()
      | 'Writing output' >> beam.io.WriteToText("data/attendance_results")
  )


!{('head -n 20 data/attendance_results-00000-of-00001')}

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


In [32]:
#word count exmaple
import apache_beam as beam
import re

#https://regex101.com/
def regex(element):
  x=element
  y = re.sub(r"[^a-zA-Z0-9]+", '', x)
  return y

p = beam.Pipeline()

word_count = (
    p
    | 'Read from file' >> beam.io.ReadFromText('data.txt')
    | 'Split words' >> beam.FlatMap(lambda x: x.split(" "))
    | 'split tabs' >> beam.FlatMap(lambda x: x.split("\t"))
    | 'trim' >> beam.Map(lambda x: x.strip()) #for safer side
    | 'remove empty elements' >> beam.Filter(lambda x: not re.match(r'^\s*$', x))
    | 'removing special characters' >> beam.Map(regex)
    | 'emitting 1 for every unique word' >> beam.Map(lambda x: (x,1))
    | 'reducing & summing' >> beam.CombinePerKey(sum)
    | 'Write output to file' >> beam.io.WriteToText("data/output_wordcount")

)

p.run()

!head -n 20 data/output_wordcount*

('KING', 243)
('LEAR', 236)
('DRAMATIS', 1)
('PERSONAE', 1)
('king', 64)
('of', 446)
('Britain', 2)
('OF', 15)
('FRANCE', 10)
('DUKE', 3)
('BURGUNDY', 8)
('CORNWALL', 63)
('ALBANY', 67)
('EARL', 2)
('KENT', 156)
('GLOUCESTER', 141)
('EDGAR', 126)
('son', 29)
('to', 429)
('Gloucester', 26)
